In [ ]:
# @title Auto-Lyrics v7
# Install necessary packages
!pip install yt-dlp
!pip install git+https://github.com/openai/whisper.git

import subprocess
import sys
from IPython.display import display, clear_output
import ipywidgets as widgets
from google.colab import output

def run_command(command):
    """Run a shell command and yield output lines in real-time."""
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in process.stdout:
        print(line, end='')
        sys.stdout.flush()  # Flush the output to force it to appear in the Colab cell
    process.wait()

def download_and_transcribe(urls):
    urls = urls.strip().split('\n')
    for i, url in enumerate(urls, 1):
        if url.strip():
            audio_file = f"audio_{i}"
            print(f"Downloading audio from: {url}")
            yt_dlp_command = f"yt-dlp -x -o '{audio_file}.%(ext)s' --force-overwrites '{url}'"
            subprocess.run(yt_dlp_command, shell=True, check=True)

            print(f"Transcribing audio: {audio_file}.opus")
            whisper_command = f"whisper {audio_file}.opus --model large"
            run_command(whisper_command)

            transcription_file = f"transcription_{i}.txt"
            print(f"Transcription completed and saved to {transcription_file}\n")

# Clear the output
clear_output(wait=True)

# Create text area widget for input
text_area = widgets.Textarea(
    value='',
    placeholder='Enter YouTube URLs separated by new lines',
    description='URLs:',
    layout={'width': '100%', 'height': '200px'}
)

# Create a button to trigger processing
process_button = widgets.Button(description="Process URLs")

# Set the button's on-click event to process the URLs
def on_button_clicked(b):
    with output.use_tags('output'):
        clear_output(wait=True)
        download_and_transcribe(text_area.value)

process_button.on_click(on_button_clicked)

# Display the widgets
display(text_area, process_button)
